<a href="https://colab.research.google.com/github/juanferEspinosa/Graph-Analytics/blob/main/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html;
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html;
!pip install -q torch-geometric;

     |████████████████████████████████| 8.0 MB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 4.9 MB/s 
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 407 kB 34.2 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
from torch.nn.modules.module import Module

import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR,StepLR

import numpy as np
import pandas as pd
import pickle as pkl
import sys
import networkx as nx
import scipy.sparse as sp
import math
import matplotlib.pyplot as plt
import time
from time import perf_counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utils functions: visualization


In [ ]:
def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(h, pos=nx.spring_layout(h, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

def normalize_adjacency_matrix(A, I):
  """
  Creating a normalized adjacency matrix with self loops.
  :param A: Sparse adjacency matrix.
  :param I: Identity matrix.
  :return A_tile_hat: Normalized adjacency matrix."""
  
  A_tilde = A + I
  degrees = A_tilde.sum(axis=0)[0].tolist()
  D = sp.diags(degrees, [0])
  D = D.power(-0.5)
  A_tilde_hat = D.dot(A_tilde).dot(D)
  return A_tilde_hat

def normalize(mx):
  """Row-normalize sparse matrix ---> Node features"""
  rowsum = np.array(mx.sum(1))
  r_inv = np.power(rowsum, -1).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  mx = r_mat_inv.dot(mx)
  return mx

def normalizemx(mx):
  """Normalization for Scattering GCN"""
  degrees = mx.sum(axis=0)[0].tolist()
  #    print(degrees)
  D = sp.diags(degrees, [0])
  D = D.power(-1)
  mx = mx.dot(D)
  return mx


def scattering1st(spmx,order):

  I_n = sp.eye(spmx.shape[0])
  adj_sct = 0.5*(spmx+I_n) # P = 1/2 * (I + WD^-1)
  adj_power = adj_sct
  adj_power = sparse_mx_to_torch_sparse_tensor(adj_power).cuda()
  adj_sct = sparse_mx_to_torch_sparse_tensor(adj_sct).cuda()
  I_n = sparse_mx_to_torch_sparse_tensor(I_n)
  if order>1:
    for i in range(order-1):
      # Generating P^(2^(k-1))
      adj_power = torch.spmm(adj_power,adj_sct.to_dense())
      print('Generating SCT')
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power)
  else:
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power.to_dense())
  return adj_int


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  """Convert a scipy sparse matrix to a torch sparse tensor."""
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)


def parse_index_file(filename):
  #Parse index file.
  index = []
  for line in open(filename):
      index.append(int(line.strip()))
  return index

def accuracy(output, labels):
  preds = output.max(1)[1].type_as(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

# Preprocessing: Importing datasets

Importing the datasets, split into training, validation and testing, normalizing it, getting the adjacency matrix, the scattering matrices, features matrix, index of nodes.

In [ ]:
def load_citation(dataset_str="cora", normalization="AugNormAdj", cuda=True):
  """  
  Load Citation Networks Datasets.
  """
  names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
  objects = []
  for i in range(len(names)):
    with open("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.{}".format(dataset_str.lower(), names[i]), 'rb') as f:
      if sys.version_info > (3, 0):
          objects.append(pkl.load(f, encoding='latin1'))
      else:
          objects.append(pkl.load(f))

  x, y, tx, ty, allx, ally, graph = tuple(objects)
  test_idx_reorder = parse_index_file("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.test.index".format(dataset_str))
  test_idx_range = np.sort(test_idx_reorder)

  if dataset_str == 'citeseer':
    # Fix citeseer dataset (there are some isolated nodes in the graph)
    # Find isolated nodes, add them as zero-vecs into the right position
    test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
    tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
    tx_extended[test_idx_range-min(test_idx_range), :] = tx
    tx = tx_extended
    ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
    ty_extended[test_idx_range-min(test_idx_range), :] = ty
    ty = ty_extended

  features = sp.vstack((allx, tx)).tolil()
  features[test_idx_reorder, :] = features[test_idx_range, :]
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
  adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
  labels = np.vstack((ally, ty))
  labels[test_idx_reorder, :] = labels[test_idx_range, :]


  idx_test = test_idx_range.tolist()
  idx_train = range(len(y))
  idx_val = range(len(y), len(y)+500)

  #   take from https://github.com/tkipf/pygcn/blob/master/pygcn/utils.py
  #    idx_train = range(140)
  #    idx_val = range(200, 500)
  #    idx_test = range(500, 1500)


  labels = torch.LongTensor(labels)
  labels = torch.max(labels, dim=1)[1]
  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)

  features = normalize(features)
  A_tilde = normalize_adjacency_matrix(adj,sp.eye(adj.shape[0]))
  adj = normalizemx(adj)
  features = torch.FloatTensor(np.array(features.todense()))
  print('Loading')
  #adj_sct1 = scattering1st(adj,1) ## psi_1 = P(I-P)
  #adj_sct2 = scattering1st(adj,2) # psi_2 = P^2(I-P^2)
  #adj_sct4 = scattering1st(adj,4) # psi_3 = P^4(I-P^4)
  adj = sparse_mx_to_torch_sparse_tensor(adj)
  A_tilde = sparse_mx_to_torch_sparse_tensor(A_tilde)
  return adj,A_tilde,features, labels, idx_train, idx_val, idx_test


In [ ]:
adj,A_tilde,features, labels, idx_train, idx_val, idx_test = load_citation()

Loading


# MODELS

First the convolutional structure is defined to finally being called in a nn Module. 

In [ ]:
class GraphAttentionLayer(nn.Module):
  """
  Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
  """
  def __init__(self, in_features, out_features, dropout, alpha, concat=True):
    super(GraphAttentionLayer, self).__init__()
    self.dropout = dropout
    self.in_features = in_features
    self.out_features = out_features
    self.alpha = alpha
    self.concat = concat

    self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
    nn.init.xavier_uniform_(self.W.data, gain=1.414)
    self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
    nn.init.xavier_uniform_(self.a.data, gain=1.414)

    self.leakyrelu = nn.LeakyReLU(self.alpha)

  def forward(self, h, adj):
    Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
    e = self._prepare_attentional_mechanism_input(Wh)
    zero_vec = -9e15*torch.ones_like(e)
    attention = torch.where(adj.to_dense() > 0, e, zero_vec)
    attention = F.softmax(attention, dim=1)
    attention = F.dropout(attention, self.dropout, training=self.training)
    h_prime = torch.matmul(attention, Wh)
    if self.concat:
        return F.elu(h_prime)
    else:
        return h_prime

  def _prepare_attentional_mechanism_input(self, Wh):
    # Wh.shape (N, out_feature)
    # self.a.shape (2 * out_feature, 1)
    # Wh1&2.shape (N, 1)
    # e.shape (N, N)
    Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
    Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
    # broadcast add
    e = Wh1 + Wh2.T
    return self.leakyrelu(e)

  def __repr__(self):
    return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [ ]:
class GAT(nn.Module):
  def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
    """Dense version of GAT."""
    super(GAT, self).__init__()
    self.dropout = dropout

    self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
    for i, attention in enumerate(self.attentions):
        self.add_module('attention_{}'.format(i), attention)

    self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)

  def forward(self, x, adj):
    x = F.dropout(x, self.dropout, training=self.training)
    # [x,y] concatenation
    x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
    x = F.dropout(x, self.dropout, training=self.training)
    x = F.elu(self.out_att(x, adj))
    return F.log_softmax(x, dim=1)

# Execution of the overall model

Hyperparameter definition, model instatiated, and training and testing

In [ ]:
from torch.autograd import Variable
torch.manual_seed(42)



epochs = 200
lr = 0.01
patience = 100
fastmode =False
cuda = torch.cuda.is_available()
model = GAT(nfeat=features.shape[1], 
                nhid=8, 
                nclass=int(labels.max()) + 1, 
                dropout=0.6, 
                nheads=8, 
                alpha=0.2)

if cuda:
    model = model.cuda()
    features = features.cuda()
    A_tilde = A_tilde.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = torch.nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=50, gamma=0.9)

features, adj, labels = Variable(features), Variable(adj), Variable(labels)

In [ ]:

import time
import glob
import os
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 430})'''))

def acc1(output, labels):
  preds = output.max(1)[1].type_as(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)
def train(epoch):
  t = time.time()
  model.train()
  optimizer.zero_grad()
  output = model(features, A_tilde)
  loss_train = F.nll_loss(output[idx_train], labels[idx_train])
  acc_train = accuracy(output[idx_train], labels[idx_train])
  loss_train.backward()
  optimizer.step()

  if not fastmode:
    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    model.eval()
    output = model(features, A_tilde)

  loss_val = F.nll_loss(output[idx_val], labels[idx_val])
  acc_val = accuracy(output[idx_val], labels[idx_val])
  print('Epoch: {:04d}'.format(epoch+1),
        'loss_train: {:.4f}'.format(loss_train.data.item()),
        'acc_train: {:.4f}'.format(acc_train.data.item()),
        'loss_val: {:.4f}'.format(loss_val.data.item()),
        'acc_val: {:.4f}'.format(acc_val.data.item()),
        'time: {:.4f}s'.format(time.time() - t))

  return loss_val.data.item()


def compute_test():
  model.eval()
  output = model(features, A_tilde)
  loss_test = F.nll_loss(output[idx_test], labels[idx_test])
  acc_test = accuracy(output[idx_test], labels[idx_test])
  print("Test set results:",
        "loss= {:.4f}".format(loss_test.data.item()),
        "accuracy= {:.4f}".format(acc_test.data.item()))

# Train model
t_total = time.time()
loss_values = []
bad_counter = 0
best = epochs + 1
best_epoch = 0
for epoch in range(epochs):
  loss_values.append(train(epoch))

  torch.save(model.state_dict(), '{}.pkl'.format(epoch))
  if loss_values[-1] < best:
    best = loss_values[-1]
    best_epoch = epoch
    bad_counter = 0
  else:
    bad_counter += 1

  if bad_counter == patience:
    break

  files = glob.glob('*.pkl')
  for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb < best_epoch:
        os.remove(file)

files = glob.glob('*.pkl')
for file in files:
  epoch_nb = int(file.split('.')[0])
  if epoch_nb > best_epoch:
      os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
compute_test()

<IPython.core.display.Javascript object>

Epoch: 0001 loss_train: 1.9529 acc_train: 0.0929 loss_val: 1.9329 acc_val: 0.4780 time: 0.1739s
Epoch: 0002 loss_train: 1.9305 acc_train: 0.2643 loss_val: 1.9197 acc_val: 0.6160 time: 0.1553s
Epoch: 0003 loss_train: 1.9029 acc_train: 0.4857 loss_val: 1.9068 acc_val: 0.6720 time: 0.1460s
Epoch: 0004 loss_train: 1.8832 acc_train: 0.5929 loss_val: 1.8936 acc_val: 0.6860 time: 0.1384s
Epoch: 0005 loss_train: 1.8666 acc_train: 0.5643 loss_val: 1.8801 acc_val: 0.7200 time: 0.1368s
Epoch: 0006 loss_train: 1.8408 acc_train: 0.6357 loss_val: 1.8663 acc_val: 0.7360 time: 0.1359s
Epoch: 0007 loss_train: 1.8209 acc_train: 0.6786 loss_val: 1.8521 acc_val: 0.7380 time: 0.1311s
Epoch: 0008 loss_train: 1.8008 acc_train: 0.6500 loss_val: 1.8376 acc_val: 0.7400 time: 0.1310s
Epoch: 0009 loss_train: 1.7905 acc_train: 0.6571 loss_val: 1.8223 acc_val: 0.7380 time: 0.1310s
Epoch: 0010 loss_train: 1.7358 acc_train: 0.7000 loss_val: 1.8058 acc_val: 0.7360 time: 0.1308s
Epoch: 0011 loss_train: 1.7367 acc_train